# **Method 3: Dynamic Client Selection with Accuracy Threshold**

This method introduces an accuracy threshold to exclude high-performing clients (accuracy ≥ 0.9) from subsequent rounds, focusing resources on weaker-performing clients.

In [ ]:
!pip install flwr
!pip install torch
!pip install matplotlib
!pip install torchvision
!pip install flwr-datasets[vision]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.2/512.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.68.1
    Uninstalling grpcio-1.68.1:
      Successfully uninstalled grpcio-1.68.1
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.3
    Uninstalling cryptography-43.0.3:
      Successfully uninstalled cryptography-43.0.3
  Attempting uninstall: typer
    Found existing installation: typer 0.15.0
    Uninstalling typer-0.15.0:
      Successfully uninstalled typer-0.15.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480

In [ ]:
!pip install -q flwr[simulation] flwr-datasets[vision] torch torchvision matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 12.5 MB/s eta 0:00:00


In [ ]:
# Import all libraries
from collections import OrderedDict
from typing import List, Tuple, Dict, Optional

import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from datasets.utils.logging import disable_progress_bar
from torch.utils.data import DataLoader

import flwr as fl
from flwr.client import Client, NumPyClient
from flwr.common import Metrics, Parameters, Scalar, FitIns, FitRes, EvaluateRes, Config
from flwr.server import ServerConfig
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy import FedAvg
from flwr_datasets import FederatedDataset


random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Device configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")


disable_progress_bar()


Training on cpu


In [ ]:
# Global constants
NUM_CLIENTS = 10 # 10 clients participate for FL
NUM_ROUNDS = 20 # Training for 20 rounds
BATCH_SIZE = 32

In [ ]:
# Function to load cifar10 datasets
def load_datasets(partition_id: int):
    fds = FederatedDataset(dataset="cifar10", partitioners={"train": NUM_CLIENTS})
    partition = fds.load_partition(partition_id)
    # Divide data on each node: 80% train, 20% test
    partition_train_test = partition.train_test_split(test_size=0.2, seed=42)
    pytorch_transforms = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )

    def apply_transforms(batch):
        batch["img"] = [pytorch_transforms(img) for img in batch["img"]]
        return batch

    # Create training and validation for each partition
    partition_train_test = partition_train_test.with_transform(apply_transforms)
    trainloader = DataLoader(
        partition_train_test["train"], batch_size=BATCH_SIZE, shuffle=True
    )
    valloader = DataLoader(partition_train_test["test"], batch_size=BATCH_SIZE)
    testset = fds.load_split("test").with_transform(apply_transforms)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    return trainloader, valloader, testloader

In [ ]:
# Model definition for CIFAR10
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
# Training function
def train(net, trainloader, epochs: int):

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for _ in range(epochs):
        for batch in trainloader:
            images, labels = batch["img"].to(DEVICE), batch["label"].to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

# Testing function
def test(net, testloader):

    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for batch in testloader:
            images, labels = batch["img"].to(DEVICE), batch["label"].to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [ ]:
# Functions to set and get model parameters
def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

In [ ]:
# Flower client implementation
class FlowerClient(NumPyClient):
    def __init__(self, net, trainloader, valloader):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config: Config) -> List[np.ndarray]:
        return get_parameters(self.net)

    def fit(self, parameters: List[np.ndarray], config: Config) -> Tuple[List[np.ndarray], int, Dict]:
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader.dataset), {}

    def evaluate(self, parameters: List[np.ndarray], config: Config) -> Tuple[float, int, Dict]:
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader.dataset), {"accuracy": float(accuracy)}

In [ ]:
# Client function
def client_fn(cid: str) -> Client:

    # Load model
    net = Net().to(DEVICE)
    # Load data
    partition_id = int(cid)
    trainloader, valloader, _ = load_datasets(partition_id)
    # Create and return the Flower client
    return FlowerClient(net, trainloader, valloader)

Strategy for Accuracy Threshold

In [ ]:
# Dynamic client selection with accuracy threshold, excludes clients above threshold and those with accuracy = 1.0
class CustomFedAvg(FedAvg):
    def __init__(self, accuracy_threshold=0.9, **kwargs):
        super().__init__(**kwargs)
        self.client_accuracies: Dict[str, float] = {}
        self.client_probabilities: Dict[str, float] = {}
        self.metrics_distributed = {"accuracy": []}
        self.latest_parameters: Optional[Parameters] = None
        self.communication_count = 0
        self.terminated = False
        self.accuracy_threshold = accuracy_threshold

    def initialize_clients(self, client_manager: ClientManager):
        for client_proxy in client_manager.all().values():
            cid = client_proxy.cid
            self.client_accuracies[cid] = 0.0
            self.client_probabilities[cid] = 1.0

    def configure_fit(
        self,
        server_round: int,
        parameters: Parameters,
        client_manager: ClientManager,
    ) -> List[Tuple[ClientProxy, FitIns]]:
        if self.terminated:
            return []

        if server_round == 1:
            self.initialize_clients(client_manager)

        all_clients = list(client_manager.all().values())
        selected_clients = []

        print(f"\nRound {server_round}:")
        for client in all_clients:
            cid = client.cid

            current_accuracy = self.client_accuracies.get(cid, 0.0)

            if current_accuracy >= self.accuracy_threshold:
                self.client_probabilities[cid] = 0.0
                print(f"  Client {cid}: accuracy {current_accuracy:.4f} >= threshold {self.accuracy_threshold}, excluded")
                continue

            # Calculate selection probability p_i(t+1) = 1 - a_i(t)
            probability = 1.0 - current_accuracy
            probability = max(0.0, probability)  # Ensuring that probability is not negative
            self.client_probabilities[cid] = probability

            x = random.uniform(0, 1)
            is_selected = x <= probability
            selection_status = "selected" if is_selected else "not selected"
            print(f"  Client {cid}: p_i({server_round}) = {probability:.4f}, x = {x:.4f}, {selection_status}")
            if is_selected:
                selected_clients.append(client)

        if not selected_clients:
            print("All clients have reached the accuracy threshold. Terminating training.")
            self.terminated = True
            return []

        self.communication_count += len(selected_clients)
        print(f"  Number of clients selected: {len(selected_clients)}")

        config = {}
        fit_ins = FitIns(parameters, config)
        return [(client, fit_ins) for client in selected_clients]

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[BaseException],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        aggregated_parameters, aggregated_metrics = super().aggregate_fit(server_round, results, failures)
        if aggregated_parameters is not None:
            self.latest_parameters = aggregated_parameters
        return aggregated_parameters, aggregated_metrics

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[BaseException],
    ) -> Optional[float]:
        metrics = []
        participating_cids = [client_proxy.cid for client_proxy, _ in results]

        for client_proxy, evaluate_res in results:
            cid = client_proxy.cid
            num_examples = evaluate_res.num_examples
            accuracy = evaluate_res.metrics.get("accuracy", 0.0)
            self.client_accuracies[cid] = accuracy
            metrics.append((num_examples, {"accuracy": accuracy}))

        print(f"\nRound {server_round} client accuracies:")
        for cid in self.client_accuracies.keys():
            accuracy = self.client_accuracies[cid]
            status = "participated" if cid in participating_cids else "did not participate"
            print(f"  Client {cid}: accuracy = {accuracy:.4f} ({status})")

        aggregated_metrics = self.weighted_average(metrics)
        global_accuracy = aggregated_metrics["accuracy"]
        self.metrics_distributed["accuracy"].append((server_round, global_accuracy))
        print(f"Round {server_round} aggregated accuracy: {global_accuracy:.4f}")
        return super().aggregate_evaluate(server_round, results, failures)

    @staticmethod
    def weighted_average(metrics: List[Tuple[int, Dict[str, Scalar]]]) -> Dict[str, Scalar]:
        accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
        examples = [num_examples for num_examples, _ in metrics]
        if sum(examples) == 0:
            return {"accuracy": 0.0}
        return {"accuracy": sum(accuracies) / sum(examples)}


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
NUM_CLIENTS = 10
NUM_ROUNDS = 20
ACCURACY_THRESHOLD = 0.9  # Adjust the threshold as needed

strategy1 = CustomFedAvg(
    accuracy_threshold=ACCURACY_THRESHOLD,
    fraction_fit=1.0,         # All clients are considered in selection
    min_fit_clients=1,        # At least one client should be available
    min_available_clients=NUM_CLIENTS,
)


In [ ]:
server_config1 = ServerConfig(num_rounds=NUM_ROUNDS)

In [ ]:
# Start the simulation
history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    client_resources={"num_gpus": 0.5 if torch.cuda.is_available() else 0},
    config=server_config1,
    strategy=strategy1,
)

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=20, no round_timeout
2024-11-20 16:37:58,991	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resource


Round 1:
  Client 12181256285627983242: p_i(1) = 1.0000, x = 0.1248, selected
  Client 1210614331923740713: p_i(1) = 1.0000, x = 0.9223, selected
  Client 15944661990935152649: p_i(1) = 1.0000, x = 0.0788, selected
  Client 6118765773027473396: p_i(1) = 1.0000, x = 0.2932, selected
  Client 17989910189733210341: p_i(1) = 1.0000, x = 0.6286, selected
  Client 10713676711731053921: p_i(1) = 1.0000, x = 0.8855, selected
  Client 3145822975221503828: p_i(1) = 1.0000, x = 0.3616, selected
  Client 10079589147956201026: p_i(1) = 1.0000, x = 0.1923, selected
  Client 16794294093924577341: p_i(1) = 1.0000, x = 0.0696, selected
  Client 8219245920631037001: p_i(1) = 1.0000, x = 0.6613, selected
  Number of clients selected: 10


(ClientAppActor pid=2088) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
(ClientAppActor pid=2087) /usr/local/lib/python3.10/dist-packages/jupyter_client/connect.py:28: DeprecationWarning: Jupyter is migrating its paths to use standard platformdirs
(ClientAppActor pid=2087) given by the platformdirs library.  To remove this warning and
(ClientAppActor pid=2087) see the appropriate new directories, set the environment variable
(ClientAppActor pid=2087) `JUPYTER_PLATFORM_DIRS=1` and then run `jupyter --paths`.
(ClientAppActor pid=2087) The use of platformdirs will be the default in `jupyter_core` v6
(ClientAppActor pid=2087)   from jupyter_core.paths import jupyter_data_dir, jupyter_runtime_dir, secure_write
(ClientAppActor pid=2087) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: 


Round 1 client accuracies:
  Client 12181256285627983242: accuracy = 0.2610 (participated)
  Client 1210614331923740713: accuracy = 0.2540 (participated)
  Client 15944661990935152649: accuracy = 0.2670 (participated)
  Client 6118765773027473396: accuracy = 0.2500 (participated)
  Client 17989910189733210341: accuracy = 0.2590 (participated)
  Client 10713676711731053921: accuracy = 0.2420 (participated)
  Client 3145822975221503828: accuracy = 0.2400 (participated)
  Client 10079589147956201026: accuracy = 0.2660 (participated)
  Client 16794294093924577341: accuracy = 0.2430 (participated)
  Client 8219245920631037001: accuracy = 0.2590 (participated)
Round 1 aggregated accuracy: 0.2541

Round 2:
  Client 12181256285627983242: p_i(2) = 0.7390, x = 0.3702, selected
  Client 1210614331923740713: p_i(2) = 0.7460, x = 0.2095, selected
  Client 15944661990935152649: p_i(2) = 0.7330, x = 0.2670, selected
  Client 6118765773027473396: p_i(2) = 0.7500, x = 0.9367, not selected
  Client 179

(ClientAppActor pid=2088) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 3x across cluster]
(ClientAppActor pid=2088)          [repeated 6x across cluster]
(ClientAppActor pid=2088)             This is a deprecated feature. It will be removed [repeated 3x across cluster]
(ClientAppActor pid=2088)             entirely in future versions of Flower. [repeated 3x across cluster]
(ClientAppActor pid=2087) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`. [repeated 2x across cluster]
(ClientAppActor pid=2087) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `

...


Round 18 client accuracies:
  Client 12181256285627983242: accuracy = 0.5550 (participated)
  Client 1210614331923740713: accuracy = 0.5250 (participated)
  Client 15944661990935152649: accuracy = 0.5750 (participated)
  Client 6118765773027473396: accuracy = 0.5290 (participated)
  Client 17989910189733210341: accuracy = 0.5460 (participated)
  Client 10713676711731053921: accuracy = 0.5510 (participated)
  Client 3145822975221503828: accuracy = 0.5630 (participated)
  Client 10079589147956201026: accuracy = 0.5800 (participated)
  Client 16794294093924577341: accuracy = 0.5500 (participated)
  Client 8219245920631037001: accuracy = 0.5180 (participated)
Round 18 aggregated accuracy: 0.5492

Round 19:
  Client 12181256285627983242: p_i(19) = 0.4450, x = 0.5806, not selected
  Client 1210614331923740713: p_i(19) = 0.4750, x = 0.9836, not selected
  Client 15944661990935152649: p_i(19) = 0.4250, x = 0.0383, selected
  Client 6118765773027473396: p_i(19) = 0.4710, x = 0.5966, not select

(ClientAppActor pid=2087)          [repeated 6x across cluster]
(ClientAppActor pid=2087) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`. [repeated 3x across cluster]
(ClientAppActor pid=2087) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 3x across cluster]
(ClientAppActor pid=2087)             This is a deprecated feature. It will be removed [repeated 3x across cluster]
(ClientAppActor pid=2087)             entirely in future versions of Flower. [repeated 3x across cluster]
INFO :      aggregate_fit: received 4 results and 0 failures
INFO :      configure_evaluate: strategy sampled 10 clients (out of 10)
(ClientAp


Round 19 client accuracies:
  Client 12181256285627983242: accuracy = 0.5710 (participated)
  Client 1210614331923740713: accuracy = 0.5380 (participated)
  Client 15944661990935152649: accuracy = 0.5750 (participated)
  Client 6118765773027473396: accuracy = 0.5270 (participated)
  Client 17989910189733210341: accuracy = 0.5590 (participated)
  Client 10713676711731053921: accuracy = 0.5490 (participated)
  Client 3145822975221503828: accuracy = 0.5480 (participated)
  Client 10079589147956201026: accuracy = 0.5780 (participated)
  Client 16794294093924577341: accuracy = 0.5560 (participated)
  Client 8219245920631037001: accuracy = 0.5330 (participated)
Round 19 aggregated accuracy: 0.5534

Round 20:
  Client 12181256285627983242: p_i(20) = 0.4290, x = 0.9489, not selected
  Client 1210614331923740713: p_i(20) = 0.4620, x = 0.9208, not selected
  Client 15944661990935152649: p_i(20) = 0.4250, x = 0.6232, not selected
  Client 6118765773027473396: p_i(20) = 0.4730, x = 0.6634, not se

(ClientAppActor pid=2087)          [repeated 10x across cluster]
(ClientAppActor pid=2088) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`. [repeated 5x across cluster]
(ClientAppActor pid=2087) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 5x across cluster]
(ClientAppActor pid=2087)             This is a deprecated feature. It will be removed [repeated 5x across cluster]
(ClientAppActor pid=2087)             entirely in future versions of Flower. [repeated 5x across cluster]
INFO :      aggregate_fit: received 2 results and 0 failures
INFO :      configure_evaluate: strategy sampled 10 clients (out of 10)
(ClientA


Round 20 client accuracies:
  Client 12181256285627983242: accuracy = 0.5630 (participated)
  Client 1210614331923740713: accuracy = 0.5320 (participated)
  Client 15944661990935152649: accuracy = 0.5660 (participated)
  Client 6118765773027473396: accuracy = 0.5350 (participated)
  Client 17989910189733210341: accuracy = 0.5430 (participated)
  Client 10713676711731053921: accuracy = 0.5370 (participated)
  Client 3145822975221503828: accuracy = 0.5460 (participated)
  Client 10079589147956201026: accuracy = 0.5660 (participated)
  Client 16794294093924577341: accuracy = 0.5580 (participated)
  Client 8219245920631037001: accuracy = 0.5280 (participated)
Round 20 aggregated accuracy: 0.5474


In [ ]:
# Communication rounds
print(f"\nTotal communication counts (number of client participations): {strategy1.communication_count}")



Total communication counts (number of client participations): 109


# **Method 4: Client Selection Based on Performance Comparison**

This method selects clients whose accuracy has not improved over previous rounds, prioritizing underperforming clients to enhance global model convergence.

In [ ]:

from typing import Dict, List, Optional, Tuple

import flwr as fl
from flwr.common import (
    Parameters,
    Scalar,
    FitIns,
    FitRes,
    EvaluateRes,
)
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy

class Aquib(fl.server.strategy.FedAvg):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.client_accuracies: Dict[str, float] = {}
        self.previous_client_accuracies: Dict[str, Optional[float]] = {}
        self.communication_count = 0
        self.terminated = False

    def initialize_clients(self, client_manager: ClientManager):
        for client_proxy in client_manager.all().values():
            cid = client_proxy.cid
            # Initialize previous accuracies to None
            self.previous_client_accuracies[cid] = None
            self.client_accuracies[cid] = 0.0

    def configure_fit(
        self,
        server_round: int,
        parameters: Parameters,
        client_manager: ClientManager,
    ) -> List[Tuple[ClientProxy, FitIns]]:
        if self.terminated:
            return []

        if server_round == 1:
            self.initialize_clients(client_manager)

        all_clients = list(client_manager.all().values())
        selected_clients = []

        print(f"\nRound {server_round}:")
        for client in all_clients:
            cid = client.cid

            current_accuracy = self.client_accuracies.get(cid, 0.0)
            previous_accuracy = self.previous_client_accuracies.get(cid)

            # Determine if the client is underperforming
            if previous_accuracy is None:
                # First round, consider all clients as underperforming
                underperforming = True
            else:
                if current_accuracy <= previous_accuracy:
                    underperforming = True
                else:
                    underperforming = False

            if underperforming:
                # Client is underperforming, select for training
                print(f"  Client {cid}: Underperforming (prev: {previous_accuracy}, current: {current_accuracy}), selected")
                selected_clients.append(client)
            else:
                # Client is performing well, do not select for training
                print(f"  Client {cid}: Performing well (prev: {previous_accuracy}, current: {current_accuracy}), not selected")

        if not selected_clients:
            print("All clients are performing well. Terminating training.")
            self.terminated = True
            return []

        self.communication_count += len(selected_clients)
        print(f"  Number of clients selected: {len(selected_clients)}")

        # Prepare fit instructions
        config = {}
        fit_ins = FitIns(parameters, config)
        return [(client, fit_ins) for client in selected_clients]

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[BaseException],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        # Call super to aggregate parameters as usual
        aggregated_parameters, aggregated_metrics = super().aggregate_fit(server_round, results, failures)
        if aggregated_parameters is not None:
            self.latest_parameters = aggregated_parameters
        return aggregated_parameters, aggregated_metrics

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[BaseException],
    ) -> Optional[float]:
        metrics = []
        participating_cids = [client_proxy.cid for client_proxy, _ in results]

        for client_proxy, evaluate_res in results:
            cid = client_proxy.cid
            num_examples = evaluate_res.num_examples
            accuracy = evaluate_res.metrics.get("accuracy", 0.0)

            # Update previous accuracy
            self.previous_client_accuracies[cid] = self.client_accuracies.get(cid, 0.0)
            # Update current accuracy
            self.client_accuracies[cid] = accuracy

            metrics.append((num_examples, {"accuracy": accuracy}))

        print(f"\nRound {server_round} client accuracies:")
        for cid in self.client_accuracies.keys():
            current_accuracy = self.client_accuracies[cid]
            previous_accuracy = self.previous_client_accuracies[cid]
            status = "participated" if cid in participating_cids else "did not participate"
            print(f"  Client {cid}: accuracy = {current_accuracy:.4f} (prev: {previous_accuracy}, {status})")

        aggregated_metrics = self.weighted_average(metrics)
        global_accuracy = aggregated_metrics["accuracy"]
        print(f"Round {server_round} aggregated accuracy: {global_accuracy:.4f}")

        return super().aggregate_evaluate(server_round, results, failures)

    @staticmethod
    def weighted_average(metrics: List[Tuple[int, Dict[str, Scalar]]]) -> Dict[str, Scalar]:
        accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
        examples = [num_examples for num_examples, _ in metrics]
        if sum(examples) == 0:
            return {"accuracy": 0.0}
        return {"accuracy": sum(accuracies) / sum(examples)}


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
NUM_CLIENTS = 10
NUM_ROUNDS = 20

strategy4 = Aquib(
    fraction_fit=1.0,         # All clients are considered in selection
    min_fit_clients=1,        # At least one client should be available
    min_available_clients=NUM_CLIENTS,
)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
server_config4 = ServerConfig(num_rounds=NUM_ROUNDS)

# Start the simulation
history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    client_resources={"num_gpus": 0.5 if torch.cuda.is_available() else 0},
    config=server_config4,
    strategy=strategy4,
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=20, no round_timeout
2024-11-22 09:48:46,850	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 7988222363.0, 


Round 1:
  Client 10892504397625728372: Underperforming (prev: None, current: 0.0), selected
  Client 15042521525871969850: Underperforming (prev: None, current: 0.0), selected
  Client 4575084854179807627: Underperforming (prev: None, current: 0.0), selected
  Client 13624414457441813354: Underperforming (prev: None, current: 0.0), selected
  Client 10344505357825835331: Underperforming (prev: None, current: 0.0), selected
  Client 15054063207148904620: Underperforming (prev: None, current: 0.0), selected
  Client 6052036123941770588: Underperforming (prev: None, current: 0.0), selected
  Client 16306295208814201252: Underperforming (prev: None, current: 0.0), selected
  Client 4372068911019174871: Underperforming (prev: None, current: 0.0), selected
  Client 6010294342599894056: Underperforming (prev: None, current: 0.0), selected
  Number of clients selected: 10


(ClientAppActor pid=9221) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
(ClientAppActor pid=9221) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=9221) 
(ClientAppActor pid=9221)             This is a deprecated feature. It will be removed
(ClientAppActor pid=9221)             entirely in future versions of Flower.
(ClientAppActor pid=9221)         
(ClientAppActor pid=9221) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
(ClientAppActor pid=9


Round 1 client accuracies:
  Client 10892504397625728372: accuracy = 0.3120 (prev: 0.0, participated)
  Client 15042521525871969850: accuracy = 0.2800 (prev: 0.0, participated)
  Client 4575084854179807627: accuracy = 0.3160 (prev: 0.0, participated)
  Client 13624414457441813354: accuracy = 0.2870 (prev: 0.0, participated)
  Client 10344505357825835331: accuracy = 0.2920 (prev: 0.0, participated)
  Client 15054063207148904620: accuracy = 0.2740 (prev: 0.0, participated)
  Client 6052036123941770588: accuracy = 0.3200 (prev: 0.0, participated)
  Client 16306295208814201252: accuracy = 0.3040 (prev: 0.0, participated)
  Client 4372068911019174871: accuracy = 0.2900 (prev: 0.0, participated)
  Client 6010294342599894056: accuracy = 0.2980 (prev: 0.0, participated)
Round 1 aggregated accuracy: 0.2973

Round 2:
  Client 10892504397625728372: Performing well (prev: 0.0, current: 0.312), not selected
  Client 15042521525871969850: Performing well (prev: 0.0, current: 0.28), not selected
  C

(ClientAppActor pid=9219) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`. [repeated 6x across cluster]
(ClientAppActor pid=9219)          [repeated 10x across cluster]
(ClientAppActor pid=9219) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 5x across cluster]
(ClientAppActor pid=9219)             This is a deprecated feature. It will be removed [repeated 5x across cluster]
(ClientAppActor pid=9219)             entirely in future versions of Flower. [repeated 5x across cluster]
(ClientAppActor pid=9219) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `

...


Round 18 client accuracies:
  Client 10892504397625728372: accuracy = 0.3120 (prev: 0.312, participated)
  Client 15042521525871969850: accuracy = 0.2800 (prev: 0.28, participated)
  Client 4575084854179807627: accuracy = 0.3160 (prev: 0.316, participated)
  Client 13624414457441813354: accuracy = 0.2870 (prev: 0.287, participated)
  Client 10344505357825835331: accuracy = 0.2920 (prev: 0.292, participated)
  Client 15054063207148904620: accuracy = 0.2740 (prev: 0.274, participated)
  Client 6052036123941770588: accuracy = 0.3200 (prev: 0.32, participated)
  Client 16306295208814201252: accuracy = 0.3040 (prev: 0.304, participated)
  Client 4372068911019174871: accuracy = 0.2900 (prev: 0.29, participated)
  Client 6010294342599894056: accuracy = 0.2980 (prev: 0.298, participated)
Round 18 aggregated accuracy: 0.2973


(ClientAppActor pid=9219) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 7x across cluster]
(ClientAppActor pid=9219)          [repeated 14x across cluster]
(ClientAppActor pid=9219)             This is a deprecated feature. It will be removed [repeated 7x across cluster]
(ClientAppActor pid=9219)             entirely in future versions of Flower. [repeated 7x across cluster]
(ClientAppActor pid=9221) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`. [repeated 8x across cluster]
INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [ROUND 20]
INFO :      configure_fit: no clients 


Round 19 client accuracies:
  Client 10892504397625728372: accuracy = 0.3120 (prev: 0.312, participated)
  Client 15042521525871969850: accuracy = 0.2800 (prev: 0.28, participated)
  Client 4575084854179807627: accuracy = 0.3160 (prev: 0.316, participated)
  Client 13624414457441813354: accuracy = 0.2870 (prev: 0.287, participated)
  Client 10344505357825835331: accuracy = 0.2920 (prev: 0.292, participated)
  Client 15054063207148904620: accuracy = 0.2740 (prev: 0.274, participated)
  Client 6052036123941770588: accuracy = 0.3200 (prev: 0.32, participated)
  Client 16306295208814201252: accuracy = 0.3040 (prev: 0.304, participated)
  Client 4372068911019174871: accuracy = 0.2900 (prev: 0.29, participated)
  Client 6010294342599894056: accuracy = 0.2980 (prev: 0.298, participated)
Round 19 aggregated accuracy: 0.2973


(ClientAppActor pid=9221) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 7x across cluster]
(ClientAppActor pid=9221)          [repeated 14x across cluster]
(ClientAppActor pid=9221)             This is a deprecated feature. It will be removed [repeated 7x across cluster]
(ClientAppActor pid=9221)             entirely in future versions of Flower. [repeated 7x across cluster]
(ClientAppActor pid=9219) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`. [repeated 7x across cluster]
(ClientAppActor pid=9219) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided 


Round 20 client accuracies:
  Client 10892504397625728372: accuracy = 0.3120 (prev: 0.312, participated)
  Client 15042521525871969850: accuracy = 0.2800 (prev: 0.28, participated)
  Client 4575084854179807627: accuracy = 0.3160 (prev: 0.316, participated)
  Client 13624414457441813354: accuracy = 0.2870 (prev: 0.287, participated)
  Client 10344505357825835331: accuracy = 0.2920 (prev: 0.292, participated)
  Client 15054063207148904620: accuracy = 0.2740 (prev: 0.274, participated)
  Client 6052036123941770588: accuracy = 0.3200 (prev: 0.32, participated)
  Client 16306295208814201252: accuracy = 0.3040 (prev: 0.304, participated)
  Client 4372068911019174871: accuracy = 0.2900 (prev: 0.29, participated)
  Client 6010294342599894056: accuracy = 0.2980 (prev: 0.298, participated)
Round 20 aggregated accuracy: 0.2973


# **Method 5: Top-K Client Selection**

This method selects a fixed number (K=3) of clients per round, either randomly or based on the lowest accuracies, to balance diversity and focus in training.

In [ ]:
# If no clients are selected select K clients for training
import random
from typing import Dict, List, Optional, Tuple

import flwr as fl
from flwr.common import (
    Parameters,
    Scalar,
    FitIns,
    FitRes,
    EvaluateRes,
)
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy

class New(fl.server.strategy.FedAvg):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.client_accuracies: Dict[str, float] = {}
        self.previous_client_accuracies: Dict[str, Optional[float]] = {}
        self.communication_count = 0
        self.terminated = False

    def initialize_clients(self, client_manager: ClientManager):
        for client_proxy in client_manager.all().values():
            cid = client_proxy.cid
            # Initialize previous accuracies to None
            self.previous_client_accuracies[cid] = None
            self.client_accuracies[cid] = 0.0

    def configure_fit(
        self,
        server_round: int,
        parameters: Parameters,
        client_manager: ClientManager,
    ) -> List[Tuple[ClientProxy, FitIns]]:
        if self.terminated:
            return []

        if server_round == 1:
            self.initialize_clients(client_manager)

        all_clients = list(client_manager.all().values())
        selected_clients = []

        print(f"\nRound {server_round}:")
        for client in all_clients:
            cid = client.cid

            current_accuracy = self.client_accuracies.get(cid, 0.0)
            previous_accuracy = self.previous_client_accuracies.get(cid)

            # Determine if the client is underperforming
            if previous_accuracy is None:
                # First round, consider all clients as underperforming
                underperforming = True
            else:
                if current_accuracy <= previous_accuracy:
                    underperforming = True
                else:
                    underperforming = False

            if underperforming:
                # Client is underperforming, select for training
                print(f"  Client {cid}: Underperforming (prev: {previous_accuracy}, current: {current_accuracy}), selected")
                selected_clients.append(client)
            else:
                # Client is performing well, do not select for training
                print(f"  Client {cid}: Performing well (prev: {previous_accuracy}, current: {current_accuracy}), not selected")

        if not selected_clients:
            print("No clients selected based on criteria.")

            # Option 1: Randomly select k clients
            k = 3  # Adjust k as needed
            selected_clients = random.sample(all_clients, k)
            print(f"  Randomly selected {k} clients.")

            # Option 2: Select top-k clients with the lowest accuracies
            # Uncomment the following lines to use Option (ii)
            """
            k = 3  # Adjust k as needed
            clients_sorted = sorted(all_clients, key=lambda c: self.client_accuracies.get(c.cid, 0.0))
            selected_clients = clients_sorted[:k]
            print(f"  Selected top-{k} clients with the lowest accuracies.")
            """

        self.communication_count += len(selected_clients)
        print(f"  Number of clients selected: {len(selected_clients)}")

        # Prepare fit instructions
        config = {}
        fit_ins = FitIns(parameters, config)
        return [(client, fit_ins) for client in selected_clients]

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[BaseException],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        # Call super to aggregate parameters as usual
        aggregated_parameters, aggregated_metrics = super().aggregate_fit(server_round, results, failures)
        if aggregated_parameters is not None:
            self.latest_parameters = aggregated_parameters
        return aggregated_parameters, aggregated_metrics

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[BaseException],
    ) -> Optional[float]:
        metrics = []
        participating_cids = [client_proxy.cid for client_proxy, _ in results]

        for client_proxy, evaluate_res in results:
            cid = client_proxy.cid
            num_examples = evaluate_res.num_examples
            accuracy = evaluate_res.metrics.get("accuracy", 0.0)

            # Update previous accuracy
            self.previous_client_accuracies[cid] = self.client_accuracies.get(cid, 0.0)
            # Update current accuracy
            self.client_accuracies[cid] = accuracy

            metrics.append((num_examples, {"accuracy": accuracy}))

        print(f"\nRound {server_round} client accuracies:")
        for cid in self.client_accuracies.keys():
            current_accuracy = self.client_accuracies[cid]
            previous_accuracy = self.previous_client_accuracies[cid]
            status = "participated" if cid in participating_cids else "did not participate"
            print(f"  Client {cid}: accuracy = {current_accuracy:.4f} (prev: {previous_accuracy}, {status})")

        aggregated_metrics = self.weighted_average(metrics)
        global_accuracy = aggregated_metrics["accuracy"]
        print(f"Round {server_round} aggregated accuracy: {global_accuracy:.4f}")

        return super().aggregate_evaluate(server_round, results, failures)

    @staticmethod
    def weighted_average(metrics: List[Tuple[int, Dict[str, Scalar]]]) -> Dict[str, Scalar]:
        accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
        examples = [num_examples for num_examples, _ in metrics]
        if sum(examples) == 0:
            return {"accuracy": 0.0}
        return {"accuracy": sum(accuracies) / sum(examples)}


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
NUM_CLIENTS = 10
NUM_ROUNDS = 20

strategy5 = New(
    fraction_fit=1.0,         # All clients are considered in selection
    min_fit_clients=1,        # At least one client should be available
    min_available_clients=NUM_CLIENTS,
)


In [ ]:
server_config5 = ServerConfig(num_rounds=NUM_ROUNDS)

# Start the simulation
history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    client_resources={"num_gpus": 0.5 if torch.cuda.is_available() else 0},
    config=server_config5,
    strategy=strategy5,
)

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=20, no round_timeout
2024-11-26 11:28:41,781	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resource


Round 1:
  Client 17902201588696226740: Underperforming (prev: None, current: 0.0), selected
  Client 11379625503450858752: Underperforming (prev: None, current: 0.0), selected
  Client 4257556833612145428: Underperforming (prev: None, current: 0.0), selected
  Client 9999993343942609094: Underperforming (prev: None, current: 0.0), selected
  Client 10987555097583255405: Underperforming (prev: None, current: 0.0), selected
  Client 13596931683633226510: Underperforming (prev: None, current: 0.0), selected
  Client 17058428006822254229: Underperforming (prev: None, current: 0.0), selected
  Client 11152687198164533792: Underperforming (prev: None, current: 0.0), selected
  Client 6744161163515617028: Underperforming (prev: None, current: 0.0), selected
  Client 11467356516771683302: Underperforming (prev: None, current: 0.0), selected
  Number of clients selected: 10


(ClientAppActor pid=18498) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
(ClientAppActor pid=18498) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=18498) 
(ClientAppActor pid=18498)             This is a deprecated feature. It will be removed
(ClientAppActor pid=18498)             entirely in future versions of Flower.
(ClientAppActor pid=18498)         
(ClientAppActor pid=18497) /usr/local/lib/python3.10/dist-packages/jupyter_client/connect.py:28: DeprecationWarning: Jupyter is migrating its paths to use standard platformdirs
(ClientAppActor pid=18497) given by the platformdirs library.  To remove this


Round 1 client accuracies:
  Client 17902201588696226740: accuracy = 0.3450 (prev: 0.0, participated)
  Client 11379625503450858752: accuracy = 0.2940 (prev: 0.0, participated)
  Client 4257556833612145428: accuracy = 0.3230 (prev: 0.0, participated)
  Client 9999993343942609094: accuracy = 0.3160 (prev: 0.0, participated)
  Client 10987555097583255405: accuracy = 0.3140 (prev: 0.0, participated)
  Client 13596931683633226510: accuracy = 0.2930 (prev: 0.0, participated)
  Client 17058428006822254229: accuracy = 0.3270 (prev: 0.0, participated)
  Client 11152687198164533792: accuracy = 0.3170 (prev: 0.0, participated)
  Client 6744161163515617028: accuracy = 0.2990 (prev: 0.0, participated)
  Client 11467356516771683302: accuracy = 0.3070 (prev: 0.0, participated)
Round 1 aggregated accuracy: 0.3135

Round 2:
  Client 17902201588696226740: Performing well (prev: 0.0, current: 0.345), not selected
  Client 11379625503450858752: Performing well (prev: 0.0, current: 0.294), not selected
 

(ClientAppActor pid=18497) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 6x across cluster]
(ClientAppActor pid=18497)          [repeated 12x across cluster]
(ClientAppActor pid=18497)             This is a deprecated feature. It will be removed [repeated 6x across cluster]
(ClientAppActor pid=18497)             entirely in future versions of Flower. [repeated 6x across cluster]
(ClientAppActor pid=18497) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`. [repeated 5x across cluster]
INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      configure_evaluate: strategy sampled 10 clients (out of 10)
(Cl

...


Round 18 client accuracies:
  Client 17902201588696226740: accuracy = 0.5540 (prev: 0.552, participated)
  Client 11379625503450858752: accuracy = 0.5320 (prev: 0.519, participated)
  Client 4257556833612145428: accuracy = 0.5690 (prev: 0.549, participated)
  Client 9999993343942609094: accuracy = 0.5200 (prev: 0.505, participated)
  Client 10987555097583255405: accuracy = 0.5470 (prev: 0.532, participated)
  Client 13596931683633226510: accuracy = 0.5240 (prev: 0.533, participated)
  Client 17058428006822254229: accuracy = 0.5420 (prev: 0.533, participated)
  Client 11152687198164533792: accuracy = 0.5740 (prev: 0.577, participated)
  Client 6744161163515617028: accuracy = 0.5470 (prev: 0.526, participated)
  Client 11467356516771683302: accuracy = 0.5180 (prev: 0.518, participated)
Round 18 aggregated accuracy: 0.5427

Round 19:
  Client 17902201588696226740: Performing well (prev: 0.552, current: 0.554), not selected
  Client 11379625503450858752: Performing well (prev: 0.519, curr

(ClientAppActor pid=18497)          [repeated 12x across cluster]
(ClientAppActor pid=18497) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 6x across cluster]
(ClientAppActor pid=18497)             This is a deprecated feature. It will be removed [repeated 6x across cluster]
(ClientAppActor pid=18497)             entirely in future versions of Flower. [repeated 6x across cluster]
(ClientAppActor pid=18498) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`. [repeated 5x across cluster]
(ClientAppActor pid=18498)          [repeated 2x across cluster]
(ClientAppActor pid=18498) WARNING :   DEPRECATED FEATURE: `client_fn` 


Round 19 client accuracies:
  Client 17902201588696226740: accuracy = 0.5500 (prev: 0.554, participated)
  Client 11379625503450858752: accuracy = 0.5460 (prev: 0.532, participated)
  Client 4257556833612145428: accuracy = 0.5650 (prev: 0.569, participated)
  Client 9999993343942609094: accuracy = 0.5190 (prev: 0.52, participated)
  Client 10987555097583255405: accuracy = 0.5330 (prev: 0.547, participated)
  Client 13596931683633226510: accuracy = 0.5300 (prev: 0.524, participated)
  Client 17058428006822254229: accuracy = 0.5370 (prev: 0.542, participated)
  Client 11152687198164533792: accuracy = 0.5780 (prev: 0.574, participated)
  Client 6744161163515617028: accuracy = 0.5310 (prev: 0.547, participated)
  Client 11467356516771683302: accuracy = 0.5190 (prev: 0.518, participated)
Round 19 aggregated accuracy: 0.5408

Round 20:
  Client 17902201588696226740: Underperforming (prev: 0.554, current: 0.55), selected
  Client 11379625503450858752: Performing well (prev: 0.532, current: 0

(ClientAppActor pid=18497) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 6x across cluster]
(ClientAppActor pid=18497)             This is a deprecated feature. It will be removed [repeated 6x across cluster]
(ClientAppActor pid=18497)             entirely in future versions of Flower. [repeated 6x across cluster]
(ClientAppActor pid=18497)          [repeated 10x across cluster]
(ClientAppActor pid=18497) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`. [repeated 5x across cluster]
(ClientAppActor pid=18497) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The pro


Round 20 client accuracies:
  Client 17902201588696226740: accuracy = 0.5710 (prev: 0.55, participated)
  Client 11379625503450858752: accuracy = 0.5380 (prev: 0.546, participated)
  Client 4257556833612145428: accuracy = 0.5560 (prev: 0.565, participated)
  Client 9999993343942609094: accuracy = 0.5190 (prev: 0.519, participated)
  Client 10987555097583255405: accuracy = 0.5410 (prev: 0.533, participated)
  Client 13596931683633226510: accuracy = 0.5330 (prev: 0.53, participated)
  Client 17058428006822254229: accuracy = 0.5360 (prev: 0.537, participated)
  Client 11152687198164533792: accuracy = 0.5940 (prev: 0.578, participated)
  Client 6744161163515617028: accuracy = 0.5460 (prev: 0.531, participated)
  Client 11467356516771683302: accuracy = 0.5220 (prev: 0.519, participated)
Round 20 aggregated accuracy: 0.5456


# **Method 6: Dynamic K and Alpha Selection**

This method dynamically adjusts client participation using K and alpha parameters to balance exploration and exploitation, optimizing accuracy and communication efficiency.

In [ ]:
# Install required packages
!pip install -q flwr[simulation] flwr-datasets[vision] torch torchvision matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.2/512.2 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.

In [ ]:
# Import Necessary Lib
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from collections import OrderedDict
from typing import List, Tuple, Dict, Optional

from datasets.utils.logging import disable_progress_bar
from torch.utils.data import DataLoader

import flwr as fl
from flwr.client import Client, NumPyClient
from flwr.common import (
    Metrics,
    Parameters,
    Scalar,
    FitIns,
    FitRes,
    EvaluateRes,
    Config,
)
from flwr.server import ServerConfig
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy import FedAvg
from flwr.common.logger import log
from logging import INFO
from flwr_datasets import FederatedDataset


In [ ]:

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Device configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")


disable_progress_bar()

Training on cpu


In [ ]:
# Global constants
NUM_CLIENTS = 10
NUM_ROUNDS = 20
BATCH_SIZE = 32
INITIAL_ROUNDS = 5  # Minimum number of rounds for initial selection
K = 3  # Number of clients to select per round
ALPHA_VALUES = [0.5, 0.2, 0.1]  # Different alpha values for experimentation

In [ ]:
# Function to load Cifar10 dataset
def load_datasets(partition_id: int):
    fds = FederatedDataset(dataset="cifar10", partitioners={"train": NUM_CLIENTS})
    partition = fds.load_partition(partition_id)
    # Divide data on each node: 80% train, 20% test
    partition_train_test = partition.train_test_split(test_size=0.2, seed=42)
    pytorch_transforms = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )

    def apply_transforms(batch):
        batch["img"] = [pytorch_transforms(img) for img in batch["img"]]
        return batch

    # Create train/val for each partition and wrap it into DataLoader
    partition_train_test = partition_train_test.with_transform(apply_transforms)
    trainloader = DataLoader(
        partition_train_test["train"], batch_size=BATCH_SIZE, shuffle=True
    )
    valloader = DataLoader(partition_train_test["test"], batch_size=BATCH_SIZE)
    testset = fds.load_split("test").with_transform(apply_transforms)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    return trainloader, valloader, testloader

In [ ]:
# Model definition for CIFAR10 same as above.
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# Training function
def train(net, trainloader, epochs: int):

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for _ in range(epochs):
        for batch in trainloader:
            images, labels = batch["img"].to(DEVICE), batch["label"].to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

# Evaluation function
def test(net, testloader):

    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for batch in testloader:
            images, labels = batch["img"].to(DEVICE), batch["label"].to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader)
    accuracy = correct / total
    return loss, accuracy

In [ ]:
# Functions to set and get model parameters
def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

In [ ]:
# Flower client implementation
class FlowerClient(NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config: Config) -> List[np.ndarray]:
        return get_parameters(self.net)

    def fit(
        self, parameters: List[np.ndarray], config: Config
    ) -> Tuple[List[np.ndarray], int, Dict]:
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader.dataset), {}

    def evaluate(
        self, parameters: List[np.ndarray], config: Config
    ) -> Tuple[float, int, Dict]:
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader.dataset), {"accuracy": float(accuracy)}

    def to_client(self) -> Client:

        return super().to_client()

In [ ]:
from flwr.common import Context
# Client function
def client_fn(cid: str) -> Client:

    # Load model
    net = Net().to(DEVICE)
    # Load data
    partition_id = int(cid)
    trainloader, valloader, _ = load_datasets(partition_id)
    # Create and return the Flower client
    return FlowerClient(cid, net, trainloader, valloader).to_client()

Strategy for Method 6

In [ ]:
from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from typing import Dict, List, Optional, Tuple
import numpy as np
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy import Strategy
from flwr.common.logger import log
from logging import INFO, WARNING, ERROR

class CS_final(Strategy):
    def __init__(
        self,
        fraction_fit: float = 1.0,
        fraction_evaluate: float = 1.0,
        min_fit_clients: int = 2,
        min_evaluate_clients: int = 2,
        min_available_clients: int = 2,
        evaluate_fn=None,
        on_fit_config_fn=None,
        on_evaluate_config_fn=None,
        accept_failures: bool = True,
        initial_rounds: int = 5,  # Minimum number of initial rounds
        k: int = 3,               # Number of clients to select per round
        alpha: float = 0.5,       # Smoothing factor for selection probability
        total_rounds: int = 20,   # Total number of rounds
    ):
        super().__init__()
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients
        self.evaluate_fn = evaluate_fn
        self.on_fit_config_fn = on_fit_config_fn
        self.on_evaluate_config_fn = on_evaluate_config_fn
        self.accept_failures = accept_failures

        self.initial_rounds = initial_rounds
        self.k = k
        self.alpha = alpha
        self.client_accuracies: Dict[str, float] = {}
        self.current_round = 0
        self.parameters: Optional[Parameters] = None  # Will be set after initialization
        self.global_accuracy: float = 0.0  # Store global model accuracy

        self.total_rounds = total_rounds  # To record communication rounds
        self.total_communications = 0     # Initialize total communications

        # Track clients selected in initial rounds to ensure all participate at least once
        self.initial_selected_clients: set = set()

    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:

        # Return None to let the server initialize parameters from a client
        return None

    def configure_fit(
        self,
        server_round: int,
        parameters: Parameters,
        client_manager: ClientManager,
    ) -> List[Tuple[ClientProxy, FitIns]]:

        self.current_round = server_round
        log(INFO, f"\n[ROUND {server_round}]")

        clients_dict = client_manager.all()
        available_cids = list(clients_dict.keys())

        # Ensure all clients have an accuracy value
        for cid in available_cids:
            if cid not in self.client_accuracies:
                self.client_accuracies[cid] = 0.0  # Default accuracy

        selected_cids = []

        if server_round <= self.initial_rounds:
            # Select top-k least accurate clients that haven't been selected yet
            # Sort clients by least accuracy
            sorted_clients = sorted(
                self.client_accuracies.items(), key=lambda x: x[1]
            )
            # Select clients not yet selected
            available_for_selection = [
                cid for cid, _ in sorted_clients if cid not in self.initial_selected_clients
            ]
            # Select up to k clients
            for cid, _ in sorted_clients:
                if cid not in self.initial_selected_clients:
                    selected_cids.append(cid)
                    self.initial_selected_clients.add(cid)
                    if len(selected_cids) == self.k:
                        break
            # If not enough unique clients, fill the rest with top-k least accurate
            if len(selected_cids) < self.k:
                additional = self.k - len(selected_cids)
                for cid, _ in sorted_clients:
                    if cid not in selected_cids:
                        selected_cids.append(cid)
                        if len(selected_cids) == self.k:
                            break
            log(
                INFO,
                f"Round {server_round}: Initial selection - Selected clients {selected_cids}",
            )
        else:
            # Probabilistic selection based on 1 - alpha * accuracy
            probabilities = {}
            for cid in available_cids:
                accuracy = self.client_accuracies[cid]
                prob = max(1 - self.alpha * accuracy, 0.0)
                probabilities[cid] = prob

            # Normalize probabilities
            total_prob = sum(probabilities.values())
            if total_prob == 0:
                # If total_prob is zero, default to uniform probabilities
                for cid in probabilities:
                    probabilities[cid] = 1.0 / len(probabilities)
            else:
                for cid in probabilities:
                    probabilities[cid] /= total_prob

            # Select k unique clients based on probabilities
            selected_cids = list(
                np.random.choice(
                    available_cids,
                    size=self.k,
                    replace=False,
                    p=[probabilities[cid] for cid in available_cids],
                )
            )
            log(
                INFO,
                f"Round {server_round}: Probabilistic selection - Selected clients {selected_cids}",
            )

        # Update total communications
        self.total_communications += len(selected_cids)

        clients = [
            clients_dict.get(cid) for cid in selected_cids if clients_dict.get(cid) is not None
        ]

        # Create fit instructions
        config = self.on_fit_config_fn(server_round) if self.on_fit_config_fn else {}
        fit_ins = FitIns(parameters, config)

        # Return client configurations as a list
        return [(client, fit_ins) for client in clients if client is not None]

    def configure_evaluate(
        self,
        server_round: int,
        parameters: Parameters,
        client_manager: ClientManager,
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:

        # Evaluate on all clients
        clients = list(client_manager.all().values())

        # Create evaluate instructions
        config = (
            self.on_evaluate_config_fn(server_round)
            if self.on_evaluate_config_fn
            else {}
        )
        evaluate_ins = EvaluateIns(parameters, config)

        # Return client configurations as a list of (ClientProxy, EvaluateIns)
        return [(client, evaluate_ins) for client in clients if client is not None]

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[BaseException],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:

        if not results:
            return None, {}

        # Total number of examples used for training
        total_examples = sum([res.num_examples for _, res in results])

        # Initialize the list to store weighted updates
        weighted_updates = []

        for _, fit_res in results:
            # Deserialize parameters to ndarrays
            client_weights = parameters_to_ndarrays(fit_res.parameters)
            # Calculate the weight based on the number of examples
            num_examples = fit_res.num_examples
            weight = num_examples / total_examples
            # Append the weighted client weights
            weighted_updates.append([layer * weight for layer in client_weights])

        # Sum the weighted updates
        aggregated_weights = [
            np.sum([update[layer] for update in weighted_updates], axis=0)
            for layer in range(len(weighted_updates[0]))
        ]

        # Serialize aggregated weights back to Parameters
        aggregated_parameters = ndarrays_to_parameters(aggregated_weights)

        # Return aggregated parameters and an empty metrics dictionary
        return aggregated_parameters, {}

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[BaseException],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:

        if not results:
            return None, {}

        # Compute weighted loss
        losses = [res.num_examples * res.loss for _, res in results]
        examples = [res.num_examples for _, res in results]
        aggregated_loss = sum(losses) / sum(examples)

        # Compute weighted accuracy
        accuracies = [
            res.num_examples * res.metrics.get("accuracy", 0.0)
            for _, res in results
        ]
        aggregated_accuracy = sum(accuracies) / sum(examples)
        self.global_accuracy = aggregated_accuracy  # Store global accuracy

        # Update client accuracies
        for client_proxy, eval_res in results:
            cid = client_proxy.cid  # Correctly retrieve the client ID
            accuracy = eval_res.metrics.get("accuracy", 0.0)
            prev_accuracy = self.client_accuracies.get(cid, 0.0)
            self.client_accuracies[cid] = accuracy

            # Log client accuracies
            log(
                INFO,
                f"Client {cid}: accuracy = {accuracy:.4f} (prev: {prev_accuracy:.4f})",
            )

        # Log aggregated accuracy
        log(
            INFO,
            f"Round {server_round} aggregated accuracy: {aggregated_accuracy:.4f}\n",
        )

        # Print total communications after the last round
        if server_round == self.total_rounds:
            log(
                INFO,
                f"[SUMMARY]\nTotal communication rounds: {self.total_communications}",
            )

        # Return aggregated loss and metrics
        return aggregated_loss, {"accuracy": aggregated_accuracy}

    def evaluate(
        self,
        server_round: int,
        parameters: Parameters,
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Evaluate model parameters using an optional validation function."""
        if self.evaluate_fn is None:

            return None
        return self.evaluate_fn(server_round, parameters)


In [ ]:
# Start simulation
if __name__ == "__main__":
    # Iterate over different alpha values for experimentation
    for alpha in ALPHA_VALUES:
        print(f"\nStarting simulation with alpha = {alpha}\n{'='*40}")

        # Create strategy instance
        strategy_strategy_final = CS_final(
            fraction_fit=1.0,
            fraction_evaluate=1.0,
            min_fit_clients=2,
            min_evaluate_clients=2,
            min_available_clients=2,
            initial_rounds=INITIAL_ROUNDS,
            k=K,
            alpha=alpha,
            total_rounds=20,
        )

        # Start the simulation
        fl.simulation.start_simulation(
            client_fn=client_fn,
            num_clients=NUM_CLIENTS,
            config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
            strategy=strategy_strategy_final,
        )

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=20, no round_timeout



Starting simulation with alpha = 0.5


2024-12-05 12:52:32,217	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 7982117684.0, 'object_store_memory': 3991058841.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=2634) 2024-12-05 12:52:37.307085: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=2634) 2024-12-05 12:52:37.392692: E external/local_xla/xla/str


Starting simulation with alpha = 0.2


2024-12-05 13:00:27,829	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'CPU': 2.0, 'memory': 8012967936.0, 'object_store_memory': 4006483968.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=7104) 2024-12-05 13:00:32.379850: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=7104) 2024-12-05 13:00:32.428236: E external/local_xla/xla/str


Starting simulation with alpha = 0.1


2024-12-05 13:08:15,917	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 8015069184.0, 'object_store_memory': 4007534592.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=11536) 2024-12-05 13:08:22.188609: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=11536) 2024-12-05 13:08:22.231654: E external/local_xla/xla/s

In [ ]:
# Start simulation
if __name__ == "__main__":
    # Set desired k and alpha values
    K = 3
    ALPHA = 0.5  # Change alpha value as needed

    print(f"\nStarting simulation with k = {K} and alpha = {ALPHA}\n{'='*50}")

    # Create strategy instance
    strategy_strategy_final = CS_final(
        fraction_fit=1.0,
        fraction_evaluate=1.0,
        min_fit_clients=2,
        min_evaluate_clients=2,
        min_available_clients=2,
        initial_rounds=INITIAL_ROUNDS,
        k=K,
        alpha=ALPHA,
        total_rounds=20,
    )

    # Start the simulation
    fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=NUM_CLIENTS,
        config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
        strategy=strategy_strategy_final,
    )


	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=20, no round_timeout



Starting simulation with k = 3 and alpha = 0.5


2024-12-05 17:09:42,887	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'object_store_memory': 3992028364.0, 'memory': 7984056731.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=1818) 2024-12-05 17:09:49.276976: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=1818) 2024-12-05 17:09:49.320655: E external/local_xla/xla/str

# **Visualisation**
For comparing accuracy over communication round for variations of K and Alpha values.

In [ ]:
import itertools
import csv
import flwr as fl
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy import Strategy
from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from typing import Dict, List, Optional, Tuple
import numpy as np
from flwr.common.logger import log
from logging import INFO, WARNING, ERROR
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Define variables
NUM_CLIENTS = 10
INITIAL_ROUNDS = 5
NUM_ROUNDS = 20

class CS_final(Strategy):
    def __init__(
        self,
        fraction_fit: float = 1.0,
        fraction_evaluate: float = 1.0,
        min_fit_clients: int = 2,
        min_evaluate_clients: int = 2,
        min_available_clients: int = 2,
        evaluate_fn=None,
        on_fit_config_fn=None,
        on_evaluate_config_fn=None,
        accept_failures: bool = True,
        initial_rounds: int = 5,  # Minimum number of initial rounds
        k: int = 3,               # Number of clients to select per round
        alpha: float = 0.5,       # Smoothing factor for selection probability
        total_rounds: int = 20,   # Total number of rounds
    ):
        super().__init__()
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients
        self.evaluate_fn = evaluate_fn
        self.on_fit_config_fn = on_fit_config_fn
        self.on_evaluate_config_fn = on_evaluate_config_fn
        self.accept_failures = accept_failures

        self.initial_rounds = initial_rounds
        self.k = k
        self.alpha = alpha
        self.client_accuracies: Dict[str, float] = {}
        self.current_round = 0
        self.parameters: Optional[Parameters] = None  # Will be set after initialization
        self.global_accuracy: float = 0.0  # Store global model accuracy

        self.total_rounds = total_rounds
        self.total_communications = 0

        # Track clients selected in initial rounds to ensure all participate at least once
        self.initial_selected_clients: set = set()

    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        # Return None to let the server initialize parameters from a client
        return None

    def configure_fit(
        self,
        server_round: int,
        parameters: Parameters,
        client_manager: ClientManager,
    ) -> List[Tuple[ClientProxy, FitIns]]:
        """Configure the next round of training."""
        self.current_round = server_round
        log(INFO, f"\n[ROUND {server_round}]")

        clients_dict = client_manager.all()
        available_cids = list(clients_dict.keys())

        # Ensure all clients have an accuracy value
        for cid in available_cids:
            if cid not in self.client_accuracies:
                self.client_accuracies[cid] = 0.0  # Default accuracy

        selected_cids = []

        if server_round <= self.initial_rounds:
            # Select top-k least accurate clients that haven't been selected yet
            # Sort clients by least accuracy
            sorted_clients = sorted(
                self.client_accuracies.items(), key=lambda x: x[1]
            )
            # Select clients not yet selected
            available_for_selection = [
                cid for cid, _ in sorted_clients if cid not in self.initial_selected_clients
            ]
            # Select up to k clients
            for cid, _ in sorted_clients:
                if cid not in self.initial_selected_clients:
                    selected_cids.append(cid)
                    self.initial_selected_clients.add(cid)
                    if len(selected_cids) == self.k:
                        break
            # If not enough unique clients, fill the rest with top-k least accurate
            if len(selected_cids) < self.k:
                additional = self.k - len(selected_cids)
                for cid, _ in sorted_clients:
                    if cid not in selected_cids:
                        selected_cids.append(cid)
                        if len(selected_cids) == self.k:
                            break
            log(
                INFO,
                f"Round {server_round}: Initial selection - Selected clients {selected_cids}",
            )
        else:
            # Probabilistic selection based on 1 - alpha * accuracy
            probabilities = {}
            for cid in available_cids:
                accuracy = self.client_accuracies[cid]
                prob = max(1 - self.alpha * accuracy, 0.0)
                probabilities[cid] = prob

            # Normalize probabilities
            total_prob = sum(probabilities.values())
            if total_prob == 0:
                # If total_prob is zero, default to uniform probabilities
                for cid in probabilities:
                    probabilities[cid] = 1.0 / len(probabilities)
            else:
                for cid in probabilities:
                    probabilities[cid] /= total_prob

            # Select k unique clients based on probabilities
            selected_cids = list(
                np.random.choice(
                    available_cids,
                    size=self.k,
                    replace=False,
                    p=[probabilities[cid] for cid in available_cids],
                )
            )
            log(
                INFO,
                f"Round {server_round}: Probabilistic selection - Selected clients {selected_cids}",
            )

        # Update total communications
        self.total_communications += len(selected_cids)

        clients = [
            clients_dict.get(cid) for cid in selected_cids if clients_dict.get(cid) is not None
        ]

        # Create fit instructions
        config = self.on_fit_config_fn(server_round) if self.on_fit_config_fn else {}
        fit_ins = FitIns(parameters, config)

        # Return client configurations as a list of (ClientProxy, FitIns)
        return [(client, fit_ins) for client in clients if client is not None]

    def configure_evaluate(
        self,
        server_round: int,
        parameters: Parameters,
        client_manager: ClientManager,
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""
        # Evaluate on all clients
        clients = list(client_manager.all().values())

        # Create evaluate instructions
        config = (
            self.on_evaluate_config_fn(server_round)
            if self.on_evaluate_config_fn
            else {}
        )
        evaluate_ins = EvaluateIns(parameters, config)

        # Return client configurations as a list of (ClientProxy, EvaluateIns)
        return [(client, evaluate_ins) for client in clients if client is not None]

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[BaseException],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate fit results using weighted average."""
        if not results:
            return None, {}


        total_examples = sum([res.num_examples for _, res in results])

        # Initialize the list to store weighted updates
        weighted_updates = []

        for _, fit_res in results:
            # Deserialize parameters to ndarrays
            client_weights = parameters_to_ndarrays(fit_res.parameters)
            # Calculate the weight based on the number of examples
            num_examples = fit_res.num_examples
            weight = num_examples / total_examples
            # Append the weighted client weights
            weighted_updates.append([layer * weight for layer in client_weights])

        # Sum the weighted updates
        aggregated_weights = [
            np.sum([update[layer] for update in weighted_updates], axis=0)
            for layer in range(len(weighted_updates[0]))
        ]

        # Serialize aggregated weights back to Parameters
        aggregated_parameters = ndarrays_to_parameters(aggregated_weights)

        # Return aggregated parameters and an empty metrics dictionary
        return aggregated_parameters, {}

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[BaseException],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation results."""
        if not results:
            return None, {}

        # Compute weighted loss
        losses = [res.num_examples * res.loss for _, res in results]
        examples = [res.num_examples for _, res in results]
        aggregated_loss = sum(losses) / sum(examples)

        # Compute weighted accuracy
        accuracies = [
            res.num_examples * res.metrics.get("accuracy", 0.0)
            for _, res in results
        ]
        aggregated_accuracy = sum(accuracies) / sum(examples)
        self.global_accuracy = aggregated_accuracy  # Store global accuracy

        # Update client accuracies
        for client_proxy, eval_res in results:
            cid = client_proxy.cid  # Correctly retrieve the client ID
            accuracy = eval_res.metrics.get("accuracy", 0.0)
            prev_accuracy = self.client_accuracies.get(cid, 0.0)
            self.client_accuracies[cid] = accuracy

            # Log client accuracies
            log(
                INFO,
                f"Client {cid}: accuracy = {accuracy:.4f} (prev: {prev_accuracy:.4f})",
            )

        # Log aggregated accuracy
        log(
            INFO,
            f"Round {server_round} aggregated accuracy: {aggregated_accuracy:.4f}\n",
        )

        # Print total communications after the last round
        if server_round == self.total_rounds:
            log(
                INFO,
                f"[SUMMARY]\nTotal communication rounds: {self.total_communications}",
            )

        # Return aggregated loss and metrics
        return aggregated_loss, {"accuracy": aggregated_accuracy}

    def evaluate(
        self,
        server_round: int,
        parameters: Parameters,
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Evaluate model parameters using an optional validation function."""
        if self.evaluate_fn is None:
            # No validation function provided
            return None
        return self.evaluate_fn(server_round, parameters)

In [ ]:
def run_simulation(k: int, alpha: float, initial_rounds: int, total_rounds: int) -> Dict:
    """
    Runs a single federated learning simulation with given k and alpha.

    Args:
        k (int): Number of clients to select per round.
        alpha (float): Smoothing factor for selection probability.
        initial_rounds (int): Number of initial rounds to enforce top-k selection.
        total_rounds (int): Total number of communication rounds.

    Returns:
        Dict: A dictionary containing k, alpha, final_accuracy, and total_communications.
    """
    print(f"\nRunning simulation with k={k}, alpha={alpha}")

    # Initialize the strategy with current k and alpha
    strategy = CS_final(
        fraction_fit=1.0,
        fraction_evaluate=1.0,
        min_fit_clients=2,
        min_evaluate_clients=2,
        min_available_clients=2,
        initial_rounds=initial_rounds,
        k=k,
        alpha=alpha,
        total_rounds=total_rounds,
    )

    # Start the Flower simulation
    fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=NUM_CLIENTS,
        config=fl.server.ServerConfig(num_rounds=total_rounds),
        strategy=strategy,
    )

    # After simulation, get the metrics
    final_accuracy = strategy.global_accuracy
    total_communications = strategy.total_communications

    print(f"Completed simulation with k={k}, alpha={alpha}: Accuracy={final_accuracy}, Communications={total_communications}")

    return {
        'k': k,
        'alpha': alpha,
        'final_accuracy': final_accuracy,
        'total_communications': total_communications
    }

Plot the graph

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

def main():
    # Define the range of k and alpha values
    #k_values = [2, 3, 5, 7]
    #alpha_values = [0.1, 0.2, 0.5, 0.8]

    k_values = [2, 3, 5, 7]
    alpha_values = [0.1, 0.2, 0.5, 0.8]

    # Initialize a list to store experiment results
    experiment_results = []

    # Iterate over all combinations of k and alpha
    for k, alpha in itertools.product(k_values, alpha_values):
        result = run_simulation(
            k=k,
            alpha=alpha,
            initial_rounds=INITIAL_ROUNDS,
            total_rounds=NUM_ROUNDS
        )
        experiment_results.append(result)

    # Save the results to a experiment_results.csv file
    csv_file = 'experiment_results.csv'
    df = pd.DataFrame(experiment_results)
    df.to_csv(csv_file, index=False)
    print(f"\nAll experiments completed. Results saved to {csv_file}")

    # visualization
    visualize_results(df)

def visualize_results(df: pd.DataFrame):


    # Create a combined label for legend entries
    df['params'] = df.apply(lambda row: f"k={row['k']}, alpha={row['alpha']}", axis=1)

    # scatter plot
    fig = px.scatter(
        df,
        x='final_accuracy',
        y='total_communications',
        color='params',
        hover_data={
            'k': True,
            'alpha': True,
            'final_accuracy': ':.4f',
            'total_communications': True
        },
        labels={
            'final_accuracy': 'Final Accuracy',
            'total_communications': 'Total Communication Rounds',
            'params': 'Parameters'
        },
        title='Federated Learning Performance: Accuracy vs Communication Rounds'
    )


    fig.update_traces(
        marker=dict(
            size=12,
            opacity=0.8,
            line=dict(width=1, color='DarkSlateGrey')
        )
    )


    fig.update_layout(
        legend_title_text='Parameters',
        xaxis_title='Final Accuracy',
        yaxis_title='Total Communication Rounds',
        hovermode='closest',
        margin=dict(l=60, r=200, t=60, b=60),
        legend=dict(
            x=1.05,
            y=1,
            xanchor='left',
            yanchor='top',
            bgcolor='rgba(255,255,255,0.9)',
            bordercolor='rgba(0,0,0,0.1)',
            borderwidth=1
        )
    )

    # Save to HTML and show
    fig.write_html('interactive_plot.html')
    fig.show()


if __name__ == "__main__":
    main()

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=20, no round_timeout



Running simulation with k=2, alpha=0.1


2024-12-12 12:59:39,138	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'object_store_memory': 4010010624.0, 'memory': 8020021248.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=2295) 2024-12-12 12:59:43.508650: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=2295) 2024-12-12 12:59:43.557216: E external/local_xla/xla/str

Completed simulation with k=2, alpha=0.1: Accuracy=0.5369, Communications=40

Running simulation with k=2, alpha=0.2


2024-12-12 13:07:37,290	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'object_store_memory': 4009637068.0, 'memory': 8019274139.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=6593) 2024-12-12 13:07:43.474775: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=6593) 2024-12-12 13:07:43.531502: E external/local_xla/xla/str

...

Completed simulation with k=7, alpha=0.2: Accuracy=0.555, Communications=140

Running simulation with k=7, alpha=0.5


2024-12-12 15:42:11,992	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0, 'memory': 8001141966.0, 'object_store_memory': 4000570982.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=79454) 2024-12-12 15:42:16.272676: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=79454) 2024-12-12 15:42:16.315071: E external/local_xla/xla/s

Completed simulation with k=7, alpha=0.5: Accuracy=0.5753, Communications=140

Running simulation with k=7, alpha=0.8


2024-12-12 15:57:30,991	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0, 'object_store_memory': 3992122982.0, 'memory': 7984245966.0, 'CPU': 2.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=86430) 2024-12-12 15:57:37.501310: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=86430) 2024-12-12 15:57:37.541004: E external/local_xla/xla/s

Completed simulation with k=7, alpha=0.8: Accuracy=0.569, Communications=140

All experiments completed. Results saved to experiment_results.csv
